In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import resource
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
# ------- Define modular methods for the task
def log_max_mem_usage():
    print(
        "Current all-time max memory: {} MB".format(
            resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000
        )
    )

test.csv
train.csv



In [2]:
train_df = pd.read_csv('../input/train.csv')
train_df.dropna(inplace=True)  # For id: qid2 174364

train_df.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
log_max_mem_usage()

Current all-time max memory: 224.752 MB


In [4]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion

# featurizers = [
#     ('char_tfidf', TfidfVectorizer(analyzer='char', ngram_range=(2, 3))),
#     ('word_tfidf', TfidfVectorizer(analyzer='word', ngram_range=(1, 2)))
# ]

# char_weight = 0.4
# combined_featurizers = FeatureUnion(
#     featurizers,
#     n_jobs=7,
#     transformer_weights={
#         'char_tfidf': char_weight,
#         'word_tfidf': 1 - char_weight
#     }
# )

unique_questions = pd.Series(pd.concat([train_df.question1, train_df.question2]).unique())
# combined_featurizers.fit(unique_questions)

char_tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))  # featurizers[0][1]
word_tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2))  # featurizers[1][1]
char_tfidf.fit(unique_questions)
word_tfidf.fit(unique_questions)

log_max_mem_usage()

Current all-time max memory: 1584.248 MB
CPU times: user 1min 25s, sys: 1.61 s, total: 1min 26s
Wall time: 1min 24s


In [9]:
from nltk.corpus import stopwords

stops = set(stopwords.words("english"))

def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    
def get_score(q1, q2):
    qset = [q1, q2]
    cv1, cv2 = char_tfidf.transform(qset)
    wv1, wv2 = word_tfidf.transform(qset)
    # cwv1, cwv2 = combined_featurizers.transform(qset)
    cv=np.dot(cv1, cv2.T)
    wv=np.dot(wv1, wv2.T)
    
    
    return dict(
        cv=np.dot(cv1, cv2.T).data[0] if cv else 0,
        wv=np.dot(wv1, wv2.T).data[0] if wv else 0,
        length_diff=abs(len(q1) - len(q2)),
        word_num_diff=abs(len(q1.split()) - len(q2.split())),
        # cwv=np.dot(cwv1, cwv2.T).data[0],
    )

def score_row(row):
    _, row = row
    q1 = row.question1.lower()
    q2 = row.question2.lower()
    if not all([set(q1.split()).difference(stops), set(q2.split()).difference(stops)]):
        return 0

    return get_score(q1, q2)

In [10]:
%%time
from sklearn.model_selection import train_test_split

scores_data = []
samp = train_df

for row in samp.iterrows():
    if row[0] % 5000 == 0:
        print(row[0])

    scores_data.append(score_row(row))

features = pd.DataFrame(scores_data)

X_train, X_test, y_train, y_test = train_test_split(features, samp.is_duplicate, test_size=0.7)
log_max_mem_usage()

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000


In [192]:
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, roc_auc_score

char_lm_model = LogisticRegression(C=100)
word_lm_model = LogisticRegression(C=1)
length_diff_lm_model = LogisticRegression(C=1)
word_num_diff_lm_model = LogisticRegression(C=1)
rf_model = RandomForestClassifier(n_estimators=100, min_samples_leaf=2, min_samples_split=3, n_jobs=-1)

def fit_models(X, y):
    rf_model.fit(X_train, y_train)
    char_lm_model.fit(X_train.cv.values.reshape(-1, 1), y_train)
    word_lm_model.fit(X_train.wv.values.reshape(-1, 1), y_train)
    length_diff_lm_model.fit(X_train.length_diff.values.reshape(-1, 1), y_train)
    word_num_diff_lm_model.fit(X_train.word_num_diff.values.reshape(-1, 1), y_train)

def predict(X):
    weights = dict(zip(X.columns, rf_model.feature_importances_))
    char_pred = char_lm_model.predict_proba(X.cv.values.reshape(-1, 1))[:, 1] # * weights['cv']
    word_pred = word_lm_model.predict_proba(X.wv.values.reshape(-1, 1))[:, 1] # * weights['wv']
    length_diff_pred = length_diff_lm_model.predict_proba(X.length_diff.values.reshape(-1, 1))[:, 1] # * weights['length_diff']
    word_num_diff_pred = word_num_diff_lm_model.predict_proba(X.word_num_diff.values.reshape(-1, 1))[:, 1] # * weights['word_num_diff']
    rf_pred = rf_model.predict_proba(X)[:, 1]

    return [char_pred, word_pred, length_diff_pred, word_num_diff_pred, rf_pred]

In [193]:
%%time
fit_models(X_train, y_train)

In [194]:
log_loss(y_test, np.mean(predict(X_test), axis=0))

In [195]:
print(roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]))
# log_loss(y_test, lm.predict_proba(X_test))
sum((y_test - (1 * (rf_model.predict_proba(X_test)[:, 1] < 0.5))) != 0)

In [196]:
list(zip(X_test.columns, rf_model.feature_importances_))

In [ ]:
from IPython.display import FileLink
test_df = pd.read_csv('../input/test.csv')

scores_data = []
samp = train_df

for row in samp.iterrows():
    if row[0] % 5000 == 0:
        print(row[0])

    scores_data.append(score_row(row))

X = pd.DataFrame(scores_data)
is_duplicate_test = np.mean(predict(X), axis=0)

log_max_mem_usage()

In [132]:
for i in zip(y_test, (1 * (lm.predict_proba(X_test)[:, 1] < 0.45))):
    print(i)

In [ ]:
train_df.ix[0].question1, train_df.ix[0].question2, train_df.ix[0].is_duplicate

In [ ]:
%%time
X_train = combined_featurizers.transform(unique_questions)

def get_feature_vector(q, X, unique_q):
    return X[np.where(unique_q == q)[0][0]]

In [ ]:
%%time
samp = train_df.head(1000).copy()
samp['q1vec'] = samp.question1.map(lambda x: get_feature_vector(x, X_train, unique_questions))
samp['q2vec'] = samp.question2.map(lambda x: get_feature_vector(x, X_train, unique_questions))

samp.groupby('is_duplicate').count()

In [ ]:
%%time
pred = []
for i, j in zip(samp.q1vec, samp.q2vec):
    pred.append(np.dot(i, j.T))

In [ ]:
roc_auc_score(samp.is_duplicate, pred)